### <b style="color:crimson">Libraries Import</b>

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from pythainlp import word_tokenize
import math
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
import os

### <b style="color:aqua">Data Preparation</b>

#### <b style="color:#008B43">Google Sheet API</b>

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name("iris-kpgdx-assessment-5526efd85250.json", scope)
client = gspread.authorize(credentials)

spreadsheet_name = 'KPGDX ประเมินความสามารถที่จำเป็นต่อความสำเร็จในการทำงาน 24 Jul - 4 Aug 2023'
spreadsheet = client.open(spreadsheet_name)

worksheet_name = 'รายชื่อผู้ประเมิน 7736'
worksheet = spreadsheet.worksheet(worksheet_name)

data = worksheet.get_all_values()

df = pd.DataFrame(data)

df.columns = df.iloc[0]
df.columns = df.columns.str.replace('\n', ' ')
df = df.iloc[1:]

excel_filename = "kpgdx_data.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Data has been saved to {excel_filename}")

Data has been saved to kpgdx_data.xlsx


#### <b style="color:#52B2BF">Filter คนที่ไม่กรอกออกไป</b>

In [3]:
import pandas as pd

df = pd.read_excel("kpgdx_data.xlsx")

df_filtered = df[df['Self Customer Centricity'].notna()]
df_filtered = df_filtered[df_filtered['Self Customer Centricity'] != '#N/A']

df_filtered.to_excel("kpgdx_filtered_data.xlsx", index=False)

print("Filtered data has been saved to kpgdx_filtered_data.xlsx")

Filtered data has been saved to kpgdx_filtered_data.xlsx


In [4]:
import pandas as pd

df = pd.read_excel("kpgdx_filtered_data.xlsx")
filter_raw_list = df.to_dict(orient='records')
for dictionary in filter_raw_list:
    for key in dictionary.keys():
        if '-comment' in key:
            if isinstance(dictionary[key], str):  # Check if value is a string before replacing
                dictionary[key] = dictionary[key].replace("_x000D_", "")


#### <b style="color:violet">Data Storage (Run ครั้งเดียวพอนะ)</b>

In [5]:
from pythainlp import word_tokenize
import math

def insert_zwnj_thai(text):
    words = word_tokenize(text)
    return "&zwnj;".join(words)

def transform_comment(value):
    if value is None or (isinstance(value, float) and math.isnan(value)) or (isinstance(value, str) and len(value) == 1):
        return '-'
    else:
        value = insert_zwnj_thai(str(value))
        value = value.replace('&zwnj;\n&zwnj;', '<br>')
        
        return value

def determine_remark(s_a_dif, s_e_dif):
    if s_a_dif >= 0 and s_e_dif >= 0:
        return 'จุดแข็งที่สามารถนำมาใช้ประโยชน์'
    elif s_a_dif >= 0 and s_e_dif < 0:
        return 'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป'
    elif s_a_dif < 0 and s_e_dif >= 0:
        return 'จุดแข็งที่สามารถพัฒนาต่อยอด'
    elif s_a_dif < 0 and s_e_dif < 0:
        return 'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก'
    else:
        return '-'

def get_learning_customer_centricity(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Customer Centricity & Transformational Leadership and Business']
        elif e_score == 4:
            return ['Customer Centricity & Transformational Leadership and Business']
        elif e_score == 3:
            return ['Customer Experience Design and Delivery']
        elif e_score == 2:
            return ['Customer Analysis, Segmentation, and Strategy']
        elif e_score == 1:
            return ['Customer Journey Design']
        else:
            return ['-']


def get_learning_data_intelligence(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Data Governance in the Digital Age']
        elif e_score == 4:
            return ['Big Data']
        elif e_score == 3:
            return ['Accelerate Growth with Big-Data and AI-Powered Solution']
        elif e_score == 2:
            return ['Data & Information Literacy, Data-Driven Strategy for Business & Innovation']
        elif e_score == 1:
            return ['Data Analysis and Insight Development', 'Data Visualization']
        else:
            return ['-']
        
def get_learning_agile_leadership(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Mastering Change Management and Transformation']
        elif e_score == 4:
            return ['Actualizing Sustainable Change']
        elif e_score == 3:
            return ['Agile 102']
        elif e_score == 2:
            return ['Agile 101']
        elif e_score == 1:
            return ['Adaptability']
        else:
            return ['-']
        
def get_learning_kpg_business_acumen(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Boosting Business Growth with Business Transformation']
        elif e_score == 4:
            return ['Business Model Innovation']
        elif e_score == 3:
            return ['Business Strategy Management (Essential)']
        elif e_score == 2:
            return ['Business Model Canvas']
        elif e_score == 1:
            return ['Business Strategy Management (Fundamental)']
        else:
            return ['-']
        
def get_learning_innovative_entrepreneurship(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Innovation and Entrepreneurship']
        elif e_score == 4:
            return ['Digital & Innovation Management (Essential)']
        elif e_score == 3:
            return ['Entrepreneurship and Initiative-Taking']
        elif e_score == 2:
            return ['Ideation Techniques for Creativity & Innovation']
        elif e_score == 1:
            return ['Digital & Innovation Management (Fundamental)']
        else:
            return ['-']
        
def get_learning_digital_dexterity(e_score, remark):
    if remark == 'จุดแข็งที่สามารถนำมาใช้ประโยชน์' or remark not in [
        'จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก',
        'จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป',
        'จุดแข็งที่สามารถพัฒนาต่อยอด'
    ]:
        return ['-']
    else:
        if e_score == 5:
            return ['Strategic Technologies Reshaping Every Business']
        elif e_score == 4:
            return ['Digital Business Strategy']
        elif e_score == 3:
            return ['Phygital: Bridging the Physical and Digital']
        elif e_score == 2:
            return ['Innovation & Digital Transformation']
        elif e_score == 1:
            return ['Innovation & Digital Transformation']
        else:
            return ['-']

sums = {}
counts = {}
data_list = []

for record in filter_raw_list:
    transformed_record = {}
    transformed_record['ID'] = record['รหัสประจำตัว Employee ID']
    transformed_record['name'] = ' '.join(record['Employee ID : Name and Surname'].split(' ')[1:])
    transformed_record['group'] = record['ชื่อหน่วยงาน (EN) Group'].replace('Group ', '')
    transformed_record['position'] = record['ชื่อตำแหน่ง (EN)']
    transformed_record['Job Level'] = record['Job Level (ใช้ในการประเมิน)']
    transformed_record['name_eng'] = record['ชื่อ-นามสกุล (EN)']
    

    
    job_level = record['Job Level (ใช้ในการประเมิน)']
    if job_level == 'EVP':
        e_score = 5
    elif job_level == 'VP':
        e_score = 4
    elif job_level in ['DM', 'SM']:
        e_score = 3
    elif job_level in ['O5', 'O6']:
        e_score = 2
    elif job_level in ['O1', 'O2', 'O3', 'O4']:
        e_score = 1
    else:
        e_score = 0
    
    for key, value in record.items():
        if "Self" in key:
            s_key = key[5:]
            if "Self-comment" not in key:
                if isinstance(value, (int, float)) and not math.isnan(value):
                    transformed_record[f's_score_{s_key}'] = value

                    sums[key] = sums.get(key, 0) + value
                    counts[key] = counts.get(key, 0) + 1

                    transformed_record[f'e_score_{s_key}'] = e_score

                    transformed_record[f's_e_dif_{s_key}'] = transformed_record[f's_score_{s_key}'] - e_score
                else:
                    transformed_record[f's_score_{s_key}'] = '-'
            else:
                transformed_record[key] = transform_comment(value)
    
    data_list.append(transformed_record)

for item in data_list:
    item['name_eng'] = item['name_eng'].replace("Mrs. ", "").replace("Miss ", "").replace("Mr. ", "").upper()

averages = {}
for key, total in sums.items():
    averages[f'a_score_{key[5:]}'] = round(total / counts[key], 2)

for record in data_list:
    for avg_key, avg_value in averages.items():
        record[avg_key] = avg_value

attributes_to_diff = [
    'Customer Centricity', 
    'Data Intelligence', 
    'Agile Leadership', 
    'KPG Business Acumen', 
    'Innovative Entrepreneurship', 
    'Digital Dexterity'
]

for record in data_list:
    for attr in attributes_to_diff:
        s_score_key = f's_score_{attr}'
        a_score_key = f'a_score_{attr}'
        diff_key = f's_a_dif_{attr}'
        remark_key = f'remark_{attr}'
        
        if isinstance(record[s_score_key], (int, float)) and isinstance(record[a_score_key], (int, float)):
            s_a_dif = round(record[s_score_key] - record[a_score_key], 2)
            s_e_dif = record[f's_e_dif_{attr}']
            record[diff_key] = s_a_dif

            record[remark_key] = determine_remark(s_a_dif, s_e_dif)

for data in data_list:
    data['learning_Customer Centricity'] = get_learning_customer_centricity(
        data['e_score_Customer Centricity'],
        data['remark_Customer Centricity']
    )
    data['learning_Data Intelligence'] = get_learning_data_intelligence(
        data['e_score_Data Intelligence'],
        data['remark_Data Intelligence']
    )
    data['learning_Agile Leadership'] = get_learning_agile_leadership(
        data['e_score_Agile Leadership'],
        data['remark_Agile Leadership']
    )
    data['learning_KPG Business Acumen'] = get_learning_kpg_business_acumen(
        data['e_score_KPG Business Acumen'],
        data['remark_KPG Business Acumen']
    )
    data['learning_Innovative Entrepreneurship'] = get_learning_innovative_entrepreneurship(
        data['e_score_Innovative Entrepreneurship'],
        data['remark_Innovative Entrepreneurship']
    )
    data['learning_Digital Dexterity'] = get_learning_digital_dexterity(
        data['e_score_Digital Dexterity'],
        data['remark_Digital Dexterity']
    )

### <b style="color:gold">Report Maker</b>

#### <b style="color:gold">Def Functions Storage</b>

In [6]:
import os

comp_list = [
        'การดำเนินธุรกิจและบริหารองค์กรโดยมีลูกค้าเป็น<br>ศูนย์กลาง (Customer Centricity)', 
        'การใช้ข้อมูลอย่างชาญฉลาดเพื่อความได้เปรียบใน<br>การแข่งขัน (Data Intelligence)', 
        'การเป็นผู้นำในแบบอไจล์ที่เปิดรับสิ่งใหม่ ๆ และ<br>ปรับตัวอย่างรวดเร็ว (Agile Leadership)',
        'ความรู้และความเฉียบคมในธุรกิจของ KPG<br>(KPG Business Acumen)',
        'การเป็นผู้ประกอบการที่ขับเคลื่อนด้วยนวัตกรรม<br>(Innovative Entrepreneurship)',
        'การใช้เทคโนโลยีดิจิทัลให้เกิดประโยชน์สูงสุดเพื่อ<br>ขับเคลื่อนกลยุทธ์ (Digital Dexterity)',
    ]

def condition_filter(x):
    if isinstance(x, str):
        if x == "จุดแข็งที่สามารถนำมาใช้ประโยชน์":
            return "background-color: #00B050; color: white"
        elif x == "จุดแข็งที่สามารถพัฒนาต่อยอด":
            return "background-color: #92D050; color: black"
        elif x == "จุดที่ต้องเร่งพัฒนาเป็นอันดับถัดไป":
                return "background-color: #FF9F9F; color: black"
        elif x == "จุดอ่อนที่ต้องเร่งพัฒนาเป็นอันดับแรก":
                return "background-color: #FF0000; color: white"
        else:
            None
    else:
        return None

def extract_data_from_list(data_list):
    dfs = []
    for data in data_list:
        rows = []
        for comp in comp_list:
            name_key = comp.split("(")[-1].split(")")[0]
            row = [
                comp,
                data.get(f's_score_{name_key}', ''),
                data.get(f'e_score_{name_key}', ''),
                data.get(f's_e_dif_{name_key}', ''),
                data.get(f'a_score_{name_key}', ''),
                data.get(f'remark_{name_key}', '')
            ]
            rows.append(row)
        df = pd.DataFrame(rows, columns=['Competency', 'S', 'E', 'S&E Difference', 'Average Score', 'Remarks'])
        dfs.append(df)
    return dfs

def get_table(dfs, condition_filter):
    tables = ''
    for index, df in enumerate(dfs):
        table_style = "margin-bottom: 30px;"
        table = f'<table class="choice-table" style="{table_style}">'
        if index == 0:
            first_column_header = "ความสามารถที่จำเป้นต่อความสำเร็จในการทำงาน"
        table += '<tr><th rowspan="2" style="border-top: 0.5px solid black; border-bottom: 1px dotted gray;">{}</th><th colspan="5" style="border-top: 0.5px solid black; border-bottom: 1px dotted gray;">ระดับความเชี่ยวชาญ</th></tr>'.format(first_column_header)
        table += '<tr><th style="border-bottom: 1px dotted gray;">ประเมิน<br>ตนเอง (S)</th><th style="border-bottom: 1px dotted gray;">องค์กร<br>คาดหวัง (E)</th><th style="border-bottom: 1px dotted gray;">ส่วนต่าง<br>S & E</th><th style="border-bottom: 1px dotted gray;">ค่าเฉลี่ย<br>(A)<sup style="font-size: 8px; vertical-align: text-bottom;">1</sup></th><th style="border-bottom: 1px dotted gray;">หมายเหตุ</th></tr>'
        
        for _, row in df.iterrows():
            style_value = condition_filter(row['Remarks'])
            table += f'<tr>'
            for i, item in enumerate(row):
                if i == len(row) - 1:
                    table += f'<td style="{style_value}">{item}</td>'
                else:
                    table += f'<td>{item}</td>'
            table += f'</tr>'
        table += '</table>'
        tables += table
    return tables

def extract_data_and_get_table(data_list, condition_filter):
    dfs = extract_data_from_list(data_list)
    return get_table(dfs, condition_filter)

def comment_data(data_list):
    dfs = []
    for data in data_list:
        rows = []
        for comp in comp_list:
            name_key = comp.split("(")[-1].split(")")[0]
            row = [
                comp,
                data.get(f'Self-comment {name_key}', '')
            ]
            rows.append(row)
        df = pd.DataFrame(rows, columns=['ความสามารถที่จำเป็นต่อความสำเร็จในการทำงาน', 'ความคิดเห็นเพิ่มเติมเพื่อสนับสนุนการประเมินของตนเอง'])
        dfs.append(df)
    return dfs

def get_comments_table_01(df):
        table = f'<table class = "comment-table">'
        table += '<thead><tr><th style="border-top: 0.5px solid black; border-bottom: 1px dotted gray;">ความสามารถที่จำเป็นต่อความสำเร็จในการทำงาน</th><th style="border-top: 0.5px solid black; border-bottom: 1px dotted gray;">ความคิดเห็นเพิ่มเติมเพื่อสนับสนุนการประเมินของตนเอง</th></tr></thead>'
        table += '<tbody>'
        for index, column in df.iterrows():
            table += '<tr><td class="left-aligned">{}</td><td class="left-aligned">{}</td></tr>'.format(
                column[0],
                column[1]
            )
        table += '</tbody>'
        table += '</table>'
        return table

def learning_data(data_list, comp_list):
    dfs = []
    for data in data_list:
        rows = []
        for comp in comp_list:
            name_key = comp.split("(")[-1].split(")")[0]
            
            learning_key = f'learning_{name_key}'
            learning_values = data.get(learning_key, '')
            
            if isinstance(learning_values, list):
                learning_values = '<ul>' + ''.join(f'<li>{item}</li>' for item in learning_values) + '</ul>'
                
            row = [comp, learning_values]
            rows.append(row)
        
        df = pd.DataFrame(rows, columns=['ความสามารถที่จำเป็นต่อความสำเร็จในการทำงาน', 'หลักสูตรออนไลน์แนะนำของไบรทเทอร์บีบนระบบ KPG Learning'])
        dfs.append(df)
    
    return dfs

def get_learning_table_01(df_list):
    all_tables = ""
    for df in df_list:
        table = f'<table class="learning-table">'
        table += '<thead><tr><th style=" border-bottom: 1px dotted gray;">ความสามารถที่จำเป็นต่อความสำเร็จในการทำงาน</th><th style="border-top: 0.5px solid black; border-bottom: 1px dotted gray;">หลักสูตรออนไลน์แนะนำของไบรทเทอร์บีบนระบบ KPG Learning</th></tr></thead>'
        
        year = 2566
        for index, (competency, courses) in enumerate(df.values):
            if index % 3 == 0:
                table += f'<tr><th colspan="2" style="border-top: 1px dotted gray; border-bottom: 1px dotted gray;"><b style="color:#010080">ภายในปี {year}</b></th></tr>'
                year += 1
            
            table += f'<tr><td class="left-aligned">{competency}</td><td class="left-aligned">{courses}</td></tr>'
        
        table += '</tbody>'
        table += '</table>'
        
        all_tables += table
    
    return all_tables

def generate_unique_filename(directory, filename):
    full_path = os.path.join(directory, filename)
    if not os.path.exists(full_path):
        return full_path
    
    filename_base, file_extension = os.path.splitext(filename)
    counter = 1
    while os.path.exists(os.path.join(directory, f"{filename_base}_{counter:02}{file_extension}")):
        counter += 1
    
    return os.path.join(directory, f"{filename_base}_{counter:02}{file_extension}")

#### <b style="color:crimson">PDF Reports</b>

In [12]:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML, CSS
import pandas as pd
import os

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("template.html")

for idx, data in enumerate(data_list):
    # ถ้าจากต้องการไฟล์ที่ 1-5 ก็พิมพ์ 0 (N-1) ข้างบน 5 ข้างล่าง
    if idx < 0: #n-1
        continue
    if idx >= 5: #n
        break

    group_name = data.get('group', 'Others')

    if not os.path.exists(group_name):
        os.makedirs(group_name)

    name = data.get('name')
    name_eng = data.get('name_eng')
    id = data.get('ID')
    comment_df = comment_data([data])[0]
    comment_table = get_comments_table_01(comment_df)
    learning_df = learning_data([data], comp_list)
    learning_table = get_learning_table_01(learning_df)
    
    html_tables = extract_data_and_get_table([data], condition_filter)
    rendered_html = template.render(tables=html_tables, name=name, comment_table=comment_table, learning_table=learning_table)

    output_filename = generate_unique_filename(group_name, f"23082023-IRIS-KPGDX-{name_eng}-ASSESSMENT-REPORT.pdf")
    HTML(string=rendered_html).write_pdf(output_filename, stylesheets=['kpgdx_page_config.css'])

    with open('kpgdx_log.txt', 'a', encoding='utf-8') as f:
        print(f"File index: {idx + 1}", file=f)
        print(f"File saved at: {output_filename}", file=f)

    print(f"File index: {idx + 1}")
    print(f"File saved at: {output_filename}")

File index: 1
File saved at: Downtown Operation\23082023-IRIS-KPGDX-NANTIRA SUKAPAK-ASSESSMENT-REPORT_11.pdf
File index: 2
File saved at: Marketing\23082023-IRIS-KPGDX-SIRINAPHANG SINAKOM-ASSESSMENT-REPORT_11.pdf
File index: 3
File saved at: Logistics\23082023-IRIS-KPGDX-THASMA KAKAEO-ASSESSMENT-REPORT_11.pdf
File index: 4
File saved at: Downtown Operation\23082023-IRIS-KPGDX-WANRADDA ROJVATANASUNTORN-ASSESSMENT-REPORT_11.pdf
File index: 5
File saved at: Chief Operating Officer\23082023-IRIS-KPGDX-ROTSUKHON LERTNOI-ASSESSMENT-REPORT_11.pdf
